# La Familia LZ de Compresores

Los algoritmos de la <b>familia LZ</b> de compresores se basan en el principio de reemplazar secuencias repetidas en el archivo por punteros a la posicion en la cual dichas secuencias fueron observadas previamente. El principio es muy simple y <b>se basa en la suposicion de que secuencias de caracteres previamente observadas pueden volver a ocurrir en el archivo</b>. Cuando esto ocurre podemos reemplazar una enorme cantidad de caracteres por un simple puntero a la posicion previa de la secuencia y su longitud. Esta tecnica, a veces llamada compresion por run-length fue popularizada a partir de 1977 por <b>Lempel</b> y <b>Ziv</b> con los algoritmos <b>LZ77</b> y <b>LZ78</b>. Estos algoritmos dieron origen a lo que se conoce como familia de compresores LZ que incluye una gran cantidad de algoritmos de compresion entre ellos <b>LZ77, LZ78, LZW, LZSS, LZRW, LZFG, LZP, LZMA, Snappy</b> y varios mas.

<img src="imagenes/lz_esquema.png">

Antes de analizar el comportamiento especifico de cada uno de los algoritmos, se puede generalizar el comportamiento de un compresor de la familia LZ de la siguiente forma:

1. Es necesario definir la longitud de un buffer o ventana que sera en donde busquemos las repeticiones. Cuando mas grande el buffer mayor cantidad de repeticiones podemos encontrar pero al mismo tiempo mas tiempo podemos tardar en procesar cada caracter. <br>
2. De acuerdo a lo anterior suele ser necesario limitar el tamanio del buffer o usar indices para poder buscar rapidamente un string dentro del mismo. <br>
3. La velocidad de la compresion en un LZ depende de la tecnica usada para buscar las repeticiones en el buffer y la longitud del mismo. <br>
4. El descompresor en todos los LZ es extremadamente rapido ya que solo necesita reemplazar las repeticiones por lo que haya en la posicion del buffer indicado. En general en velocidad de descompresion los algoritmos LZ son imposibles de derrotar. <br>
5. Cada repeticion se reemplaza por simbolos que permiten al descompresor determinar la posicion del buffer y la longitud de la repeticion.

## RLE

El primer algoritmo es extremadamente simple y sirve solo como una rapida introduccion. El metodo RLE (Run Length Encoding), tiene como unico objetivo reemplazar secuencias de caracteres iguales repetidos en el archivo por una codificacion mas compacta. Cada vez que un caracter aparezca cuatro o mas veces en el archivo lo vamos a reemplazar por los cuatro caracteres y un numero en 8 bits indicando cuantas repeticiones mas agregar. De esta forma AAAAAAAA se transforma en AAAA4, AAAA se transforma en AAAA0. Este metodo expande un byte por cada repeticion de longitud 4, no gana ni pierde con repeticiones de longitud 5 y ahorra n - 4 bytes por cada repeticion de longitud n.

ARCHIVO: ABCCCCCCCDDDFFFFAAAAAAAACD <br>
COMPRIMIDO: ABCCCC3DDDFFFF0AAAA4CD 

<b>Descomprimir</b> es muy simple solo hay que ir copiando los caracteres al archivo de salida pero si se observan cuatro caracteres iguales seguidos entonces el byte que se lee a continuacion se lo considera como un numero que indica cuantas veces mas repetir el caracter anterior. 

## LZSS

<b>LZSS</b> es una version mejorada de <b>LZ77</b> el algoritmo que dio inicio a la familia de compresores <b>LZ</b>. Al ser una version mejorada de <b>LZ77</b> en muchos casos se llama <b>LZ77</b> a <b>LZSS</b> ya que la version original no tiene uso practico. En <b>LZSS</b> se usa un bit para distinguir repeticiones de no-repeticiones. Cada vez que se comprime un caracter del archivo se busca hacia atras hasta una cierta cantidad maxima de caracteres llamada <b>ventana</b> en busca de repeticiones. Si se encuentra una repeticion entonces se emite un bit en 1 seguido de la posicion de la repeticion en la ventana y la longitud de la misma. Cuando no se encuentran repeticiones se emite un bit en 0 y luego el caracter en cuestion en 8 bits (es decir, sin comprimir).

En principio entonces hay que fijar dos parametros: la <b>longitud de la ventana</b> en la que se buscan las <b>repeticiones</b> y la <b>longitud minima de un match</b> (podria no tener sentido buscar matches de longitud 1).

Por ejemplo, se comprime <b>LZSS</b> con una longitud minima de match = 2 y ventanas de 4 bytes.

<b>ARCHIVO</b>: ABCDADADAABDAA

Para comprimir con <b>LZSS</b>, se siguen los siguientes pasos:
1. Letra A. Ventana = []. A no se encuentra en la ventana. El 0 indicando que no hay repeticion. <b>Se emite 0A</b><br>
2. Letra B. Ventana = [A]. B no se encuentra en la ventana. <b>Se emite 0B</b>.<br>
3. Letra C. Ventana = [A,B]. C no se encuentra en la ventana. <b>Se emite 0C</b>.<br>
4. Letra D. Ventana = [A,B,C]. D no se encuentra en la ventana. <b>Se emite 0D</b>.<br>
5. Letra A. Ventana = [A,B,C,D]. A se encuentra en la ventana pero sigue D y AD no se encuentra en la ventana. <b>Se emite 0A</b>.<br>
6. Letra D. Ventana = [B,C,D,A]. D se encuentra en la ventana y sigue A. Osea, tenemos DA. Y DA tiene longitud 2 y se encuentra en la ventana. Entonces, indicamos relativo a la ventana en donde esta la repeticion: es desde la posicion 1 de la ventana porque se cuenta de derecha a izquierda hasta 4 posiciones mas adelante. Por lo tanto es (1,4). <b>Se emite 1(1,4)</b>. El 1 indicando que hay repeticion. <br>
7. Letra A. Ventana = [D,A,D,A]. A se encuenta en la ventana. Sigue B y AB no se encuentra en la ventana. <b>Se emite 0A</b>. <br>
8. Letra B. Ventana = [A,D,A,A]. B no se encuentra en la ventana. <b>Se emite 0B</b>. ,<br>
9. Letra D. Ventana = [D,A,A,B]. D se encuentra en la ventana y sigue A, y sigue A. Se encuentran en la ventana. De la posicion 3 de la ventana contamos 3 para adelante, queda (3,3). <b>Se emite 1(3,3)</b>.

<b>MODELO</b>: 0A 0B 0C 0D 0A 1(1,4) 0A 0B 1(3,3)

<b>Notar</b>: Que las pociones en la ventana se numeran de derecha a izquierda, es decir, el ultimo elemento de la ventana esta en la posicion 0. Tambien se puede notar que la ventana esta limitada hacia atras pero no hacia adelante, con lo cual se podrian emitir matches de longitud 1Gb siempre y cuando esta longitud sea representable.

Para comprimir lo unico que resta es definir es como representar los pares de tipo <b>(posicion, longitud)</b>. En LZSS esto se hace simplemente mediante codigos binarios de longitud fija. Como nuestra ventan tiene 4 bytes las posiciones posibles son 0,1,2,3 por lo que se necesita 2 bits para representarlas y por lo tanto cada longitud sera un binario de 2 bits. Las longitudes deben comenzar desde 2 y tenemos que fijar una longitud maxima, si por ejemplo se decide usar 2 bits para las longitudes entoncesp odemos representar 4 longitudes el minimo es 2 esto quiere decir que las longitudes representables seran 2,3,4 y 5. Esto quiere decir que la cantidad de bits que usamos para la posicion define la longitud de la ventana de busqueda (o viceversa) y la longitud en bits que usemos para la longitud del march de fine el match maximo posible.

Si quisieramos representar en forma binaria el archivo de ejemplo, quedaria algo del siguiente estilo:

1(1,4):
- 1 bit que indica posicion.
- 01 = 1 indicando la posicion en la ventana.
- 10 = 4 indicando la longitud del match. (Recordar 00 es 2 porque es el minimo)

1(3,3):
- 1 bit que indica posicion.
- 11 = 3 indicando la posicion en la ventana.
- 01 = 3 indicando la longitud del match. (Recordar 00 es 2 porque es el minimo)

<b>ARCHIVO COMPRIMIDO</b>: 0A 0B 0C 0D 0A 10110 0A 0B 0C 11101

La <b>descompresion</b> en <b>LZSS</b> es extremadamente rapida, cuando el descompresor encuentra un bit en 0 emite los 8 bits siguientes como un caracter literal. Cuando el descompresor encuentra un bit en 1 indicando repeticion lee los siguientes 2 bits para establecer la posicion de la ventana, lee los siguientes 2 bits y suma la longitud minima de match para determinar la longitud del match y luego simplemente copia esa cantidad de bits a partir de la posicion indicada al archivo de salida. Dado que el descompresor no hace practicamente nada es extremadamente veloz.

## LZW

<b>LZW</b> es una version mejorada del algoritmo <b>LZ78</b> publicado en 1978 por <b>Lempel y Ziv</b> y, al igual que con <b>LZ77</b>, en muchos lados se llama <b>LZ78</b> a <b>LZW</b> aunque sean algoritmos diferentes. La <b>W</b> en <b>LZW</b> es por Welch quien fue el que trabajo en las mejoras al algoritmo junto con Lempel y Ziv.

La idea principal de <b>LZW</b> es la misma que la de todos los algoritmos de la familia <b>LZ</b>: reemplazar secuencias previamente observadas por un codigo que las represente. <b>LZW</b> logra esto mediante la utilizacion de una tabla o diccionario en donde va almacenando las secuencias previamente observadas en el archivo. De esta forma cuando encuentra alguna secuencia que ya vio simplemente la reemplaza por el indice de la misma en el diccionario o tabla.

Inicialemente <b>LZW</b> comienza con una tabla de 512 posiciones en donde las primeras 256 entradas ya estan llenas con las 256 combinaciones posibles de 8 bits y las restantes 256 posiciones estan vacias. Cada posicion de la tabla se representa logicamente mediante 9 bits. El algoritmo funciona entonces de la siguiente forma:

#### Algoritmo:

Por ejemplo, sea el siguiente archivo: <b>ABCABCA</b>

Se lee la A del archivo, que ya se encuentra cargada en la tabla entre alguna de las 256 primeras posiciones. En consecuencia, se lee B. El simbolo AB no esta en la tabla por lo tanto se emite el codigo A, es decir, su indice en la tabla que ocupa 9 bits y agregamos AB en la posicion 256 de la tabla (numerando desde 0).

<img src="imagenes/LZW_luego_de_comprimir_A.png">